In [ ]:
                   
print('hi')

### Args:
- in_channels (int): Number of channels in the input image
- out_channels (int): Number of channels produced by the convolution
- kernel_size (int or tuple): Size of the convolving kernel
- stride (int or tuple, optional): Stride of the convolution. Default: 1
- padding (int or tuple, optional): Zero-padding added to both sides of the input. Default: 0
- padding_mode (string, optional). Accepted values `zeros` and `circular` Default: `zeros`
- dilation (int or tuple, optional): Spacing between kernel elements. Default: 1
- groups (int, optional): Number of blocked connections from input channels to output channels. 

### Test case 1

transform

Data:
- F = F, X = Theta
- 



In [2]:
%cd ~/lab/stage/             
from src.surrogate.modules import *
from src.surrogate.deeplearning.core import *
from src.dataset import *

from sklearn import preprocessing
from torch import nn

tf1_1 = preprocessing.MinMaxScaler(feature_range=(0, 1))
tx1_1 = preprocessing.StandardScaler()

tf1_2 = preprocessing.MinMaxScaler(feature_range=(0, 1))
tx1_2 = preprocessing.StandardScaler()

tf1_3 = preprocessing.MinMaxScaler(feature_range=(0, 1))
tx1_3 = preprocessing.StandardScaler()

transform_1 = TransformDataFrame.from_ordered_dict(OrderedDict([
    ("tf1", (tf1_1, ["f1", "f2", "f3", "f4", 'f5'])),
    ("tx1", (tx1_1, [f"x{i}" for i in range(1,653)]))
]
))

transform_2 = TransformDataFrame.from_ordered_dict(OrderedDict([
    ("tf1", (tf1_2, ["f1", "f2", "f3", "f4", 'f5'])),
    ("tx1", (tx1_2, [f"x{i}" for i in range(1,301)]))
]
))
transform_3 = TransformDataFrame.from_ordered_dict(OrderedDict([
    ("tf1", (tf1_3, ["f1", "f2", "f3", "f4", 'f5'])),
    ("tx1", (tx1_3, [f"x{i}" for i in range(1,653)]))
]
))

# NB10422645 (Stage - Airbus)/data/
# ds_t1_omega = DataSetFSeries.from_parquet(root_dir="data", file_name="dataSet2_gridSampled_6_series_omega", transform=transform_1)
# ds_t2_omega = DataSetFSeries.from_parquet(root_dir="data", file_name="dataSet2_randomSampled_6_series_omega", transform=transform_2)
# ds_t3_omega = DataSetFSeries.from_parquet(root_dir="data", file_name="dataSet2_haltonSampled_6_sdp5_series_omega", transform=transform_3)

nans = lambda df: df[df.isnull().any(axis=1)]

# nans(ds_t2_omega.get_df())

# ds_t3_omega = DataSetFSeries.from_parquet(root_dir="data", file_name="dataSet2_gridSampled_6_series_omega", transform=transform_3)

/home/ggarrett/lab/stage


In [3]:
# Define training function.
def train_model(name, ds, model, model_folder, lr=1e-1, epoch=50, load=False, scheduler=None):
    trainer = Trainer(model_folder, ds, model, torch.optim.Adam, torch.nn.MSELoss, num_epochs=epoch, batch_size=int(1e2), learning_rate=lr)
    if scheduler is not None:
        trainer.add_scheduler(torch.optim.lr_scheduler.MultiStepLR, milestones=scheduler[0], gamma=scheduler[1])
    if load:
        trainer.load_checkpoint(model_folder)
    trainer.train()
    return trainer._model

class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
    
class Flatten2(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), 1, -1)
    
class Unflatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), input.size(1), 1, 1)

In [4]:
import torch
print(torch.cuda.is_available())

class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
    
class Flatten2(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), 1, -1)
    
class Unflatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), input.size(1), 1, 1)


model2 = nn.Sequential(
    
    Flatten2(),
    
    nn.Conv1d(1, 126, stride=4, kernel_size=72, padding=0),       # [64,   1, 155]
#     nn.LeakyReLU(negative_slope=0.01, inplace=True),             # 
    nn.Sigmoid(),             # 
    nn.MaxPool1d(11, stride=2, padding=0),                       # [64,   1,  73]
#     nn.Dropout(0.05),
    
    nn.Conv1d(126, 258, stride=1, kernel_size=19, padding=9),     # [128,  1,  73]
#     nn.LeakyReLU(negative_slope=0.01, inplace=True),             # 
    nn.Sigmoid(),             # 

    nn.MaxPool1d(4, stride=3, padding=0),                        # [128,  1,  24]
#     nn.Dropout(0.05),
    
    nn.Conv1d(258, 258, stride=1, kernel_size=3, padding=1),     # [172,  1,  24]
#     nn.LeakyReLU(negative_slope=0.01, inplace=True),             # 
    nn.Sigmoid(),             # 

    nn.MaxPool1d(4, stride=2, padding=0),                        # [172,  1,  11]
#     nn.Dropout(0.05),
    
    nn.Conv1d(258,126, stride=1, kernel_size=3, padding=1),      # [128,  1,  11]
#     nn.LeakyReLU(negative_slope=0.01, inplace=True),             # 
    nn.Sigmoid(),             # 

    nn.MaxPool1d(4, stride=2, padding=0),                        # [128,  1,   5]
#     nn.Dropout(0.05),
    
    Flatten(),
    
    nn.Linear(504, 504),                                       # [,860]
#     nn.LeakyReLU(negative_slope=0.01, inplace=True),             # 
    nn.Sigmoid(),             # 

    
    nn.Linear(504, 504),                                       # [,860]
#     nn.LeakyReLU(negative_slope=0.01, inplace=True),             # 
    nn.Sigmoid(),             # 

    
    nn.Linear(504, 5),
    Unflatten()
    )

print(model2)

True
Sequential(
  (0): Flatten2()
  (1): Conv1d(1, 126, kernel_size=(72,), stride=(4,))
  (2): Sigmoid()
  (3): MaxPool1d(kernel_size=11, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv1d(126, 258, kernel_size=(19,), stride=(1,), padding=(9,))
  (5): Sigmoid()
  (6): MaxPool1d(kernel_size=4, stride=3, padding=0, dilation=1, ceil_mode=False)
  (7): Conv1d(258, 258, kernel_size=(3,), stride=(1,), padding=(1,))
  (8): Sigmoid()
  (9): MaxPool1d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv1d(258, 126, kernel_size=(3,), stride=(1,), padding=(1,))
  (11): Sigmoid()
  (12): MaxPool1d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False)
  (13): Flatten()
  (14): Linear(in_features=504, out_features=504, bias=True)
  (15): Sigmoid()
  (16): Linear(in_features=504, out_features=504, bias=True)
  (17): Sigmoid()
  (18): Linear(in_features=504, out_features=5, bias=True)
  (19): Unflatten()
)


In [5]:
# name = "dataSet2_gridSampled_6_model"
# folder = f"models/{name}"

# ds = DataSetFSeries.from_parquet(root_dir="data", file_name="dataSet2_gridSampled_6_series_omega", transform=transform_1)
# model_grid = train_model("dataSet2_gridSampled_6_model", ds, model2, folder, load=False,
# #                            scheduler=([10], 0.1),
#                          epoch=50,
#                          lr=1e-4)

# # # https://open.spotify.com/track/0rUNZQuYQvOz6A6zwyT6tM?si=6WSixNxgSfW2FTLLytNB8g

In [6]:
name = "dataSet2_randomSampled_6_model"
folder = f"models/{name}"

ds = DataSetFSeries.from_parquet(root_dir="data", file_name="dataSet2_randomSampled_6_series_omega", transform=transform_2)
model_random = train_model("dataSet2_randomSampled_6_model", ds, model2, folder, load=False,
#                            scheduler=([10], 0.1),
                         epoch=50,
                         lr=1e-4)

Epoch [  1  / 50  ]  |  Train Loss:  0.009242  |  Test Loss:  0.002754  |  lr:  0.00010
Epoch [  2  / 50  ]  |  Train Loss:  0.002901  |  Test Loss:  0.002828  |  lr:  0.00010
Epoch [  3  / 50  ]  |  Train Loss:  0.002907  |  Test Loss:  0.002724  |  lr:  0.00010
Epoch [  4  / 50  ]  |  Train Loss:  0.002917  |  Test Loss:  0.002977  |  lr:  0.00010
Epoch [  5  / 50  ]  |  Train Loss:  0.002916  |  Test Loss:  0.002808  |  lr:  0.00010
Epoch [  6  / 50  ]  |  Train Loss:  0.002922  |  Test Loss:  0.002928  |  lr:  0.00010
Epoch [  7  / 50  ]  |  Train Loss:  0.002934  |  Test Loss:  0.002797  |  lr:  0.00010
Epoch [  8  / 50  ]  |  Train Loss:  0.002924  |  Test Loss:  0.002767  |  lr:  0.00010
Epoch [  9  / 50  ]  |  Train Loss:  0.002925  |  Test Loss:  0.002984  |  lr:  0.00010
Epoch [ 10  / 50  ]  |  Train Loss:  0.002905  |  Test Loss:  0.002762  |  lr:  0.00010
Epoch [ 11  / 50  ]  |  Train Loss:  0.002887  |  Test Loss:  0.002738  |  lr:  0.00010
Epoch [ 12  / 50  ]  |  Train Lo

In [7]:
# name = "dataSet2_haltonSampled_6_model"
# folder = f"models/{name}"

# ds = DataSetFSeries.from_parquet(root_dir="data", file_name="dataSet2_haltonSampled_6_sdp5_series_omega", transform=transform_3)
# model_halton = train_model("dataSet2_haltonSampled_6_model", ds, model2, folder, load=False,
# #                            scheduler=([10], 0.1),
#                          epoch=50,
#                          lr=1e-4)

In [ ]:
name = "dataSet2_sol1_gridSampled_6_model"
folder = f"models/{name}"

ds = DataSetFSeries.from_parquet(root_dir="data", file_name="dataSet2_sol1_gridSampled_6_series_omega", transform=transform_3)
model_sol1 = train_model("vdataSet2_sol1_gridSampled_6_model", ds, model2, folder, load=False,
                           scheduler=([10], 0.1),
                         epoch=50,
                         lr=1e-4)

Epoch [  1  / 50  ]  |  Train Loss:  0.011992  |  Test Loss:  0.002763  |  lr:  0.00010
Epoch [  2  / 50  ]  |  Train Loss:  0.001873  |  Test Loss:  0.001279  |  lr:  0.00010
Epoch [  3  / 50  ]  |  Train Loss:  0.001010  |  Test Loss:  0.000865  |  lr:  0.00010
Epoch [  4  / 50  ]  |  Train Loss:  0.000703  |  Test Loss:  0.000601  |  lr:  0.00010
Epoch [  5  / 50  ]  |  Train Loss:  0.000542  |  Test Loss:  0.000467  |  lr:  0.00010
Epoch [  6  / 50  ]  |  Train Loss:  0.000441  |  Test Loss:  0.000445  |  lr:  0.00010
Epoch [  7  / 50  ]  |  Train Loss:  0.000367  |  Test Loss:  0.000343  |  lr:  0.00010
Epoch [  8  / 50  ]  |  Train Loss:  0.000313  |  Test Loss:  0.000307  |  lr:  0.00010
Epoch [  9  / 50  ]  |  Train Loss:  0.000274  |  Test Loss:  0.000275  |  lr:  0.00010
Epoch [ 10  / 50  ]  |  Train Loss:  0.000248  |  Test Loss:  0.000236  |  lr:  0.00010
Epoch [ 11  / 50  ]  |  Train Loss:  0.000196  |  Test Loss:  0.000203  |  lr:  0.00001
Epoch [ 12  / 50  ]  |  Train Lo

In [ ]:
x_ = list(model_halton.children())[1].weight[0]
print(len(x_.flatten()))
print(list(model_halton.children())[1].weight[0])

In [ ]:
model_halton.eval()

# Fixing random state for reproducibility
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
count = 0
for res in list(model_halton.children())[1].weight[::1]:
# res = .weight[0]
    res2 = res.detach().cpu().numpy().flatten()
    plt.figure(dpi=400, figsize=(10,2))
    plt.axis('off')
    imgplot = plt.imshow(np.tile(res2.reshape(1,-1),(1,1)), cmap='gray')



In [ ]:
model = model_halton


# print(list(model.children())[4].weight.shape)


# for res in list(model.children())[4].weight[::2]:
# # res = .weight[0]
#     res2 = res.detach().cpu().numpy().flatten()
#     plt.figure(dpi=400, figsize=(10,2))
#     plt.axis('off')
#     imgplot = plt.imshow(np.tile(res2.reshape(1,-1),(20,1)), cmap='gray')

res = list(model.children())[13].weight.detach().cpu().numpy()[:,:,]

avg = np.mean(res, axis=1)
print(res.shape)

plt.figure(dpi=500, figsize=(10,2))
plt.axis('off')
imgplot = plt.imshow(avg, cmap='gray')
# imgplot = plt.imshow(res, cmap='gray')


# print(avg.shape)


In [ ]:
for res in avg:
# res = .weight[0]
#     res2 = res.detach().cpu().numpy().flatten()
    plt.figure(dpi=400, figsize=(10,2))
    plt.axis('off')
    imgplot = plt.imshow(np.tile(res.reshape(1,-1),(1,1)), cmap='gray')



In [ ]:
for res in list(model.children())[9].weight[::3]:
# res = .weight[0]
    res2 = res.detach().cpu().numpy().flatten()
    plt.figure(dpi=400, figsize=(10,2))
    plt.axis('off')
    imgplot = plt.imshow(np.tile(res2.reshape(1,-1),(1,1)), cmap='gray')

